### Import Required Packages and Set Options

#### Import Base Libraries

In [2]:
import os
import sys

import numpy as np
import pandas as pd
import multiprocessing as mp

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [3]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Dynamically Re-Load all Package Modules on Execution

In [4]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

#### Set File Path Constants

In [5]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [6]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Load Items Data

In [7]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Load Ratings Data

In [9]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

In [10]:
ratings_explicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,196,242,3,1997-12-04 15:55:49,0
1,186,302,3,1998-04-04 19:22:22,0
2,22,377,1,1997-11-07 07:18:36,0
3,244,51,2,1997-11-27 05:02:03,0
4,166,346,1,1998-02-02 05:33:16,0


#### Generate Implicit Feedback Ratings Data

In [11]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [12]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create (Training, Validation) Interactions for Evaluation

In [13]:
print(ratings_implicit.shape)
ratings_implicit.head()

(54194, 5)


,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


In [14]:
test_pct = 0.25
drop_users = (2, 4)

train_mask = (ratings_implicit.index / ratings_implicit.shape[0]) <  (1 - test_pct)
valid_mask = (ratings_implicit.index / ratings_implicit.shape[0]) >= (1 - test_pct)

interactions_total = ratings_implicit[['user_id', 'item_id']]
interactions_train = ratings_implicit[train_mask][['user_id', 'item_id']]
interactions_valid = ratings_implicit[valid_mask][['user_id', 'item_id']]

# drop some users from the training data to test cold-start functionality
interactions_train = interactions_train[~interactions_train.user_id.isin(drop_users)]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

print("total shape: {}".format(interactions_total.shape))
print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))
print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

total shape: (54194, 2)
train shape: (40598, 2)
valid shape: (13548, 2)
train users: 941
valid users: 905
cold-start users: {2, 4}


In [15]:
iteraction_users = interactions_train.user_id.unique()
iteraction_items = interactions_train.item_id.unique()

#### Create User/Item Features Data for Testing

In [16]:
user_features = users_df[users_df.user_id.isin(iteraction_users)]
print(user_features.shape, users_df.shape)
user_features.head()

(941, 27) (943, 27)


,user_id,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,6,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
item_features = items_df[items_df.item_id.isin(iteraction_items)]
print(item_features.shape, items_df.shape)
item_features.head()

(1421, 19) (1682, 19)


,item_id,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


### Test Out Core Package Functionality

#### Initialize the Model

In [18]:
model = RankFM(factors=10, learning_rate=0.1, regularization=0.01, sigma=0.1)
model

#### Initialize and Inspect Internal Data

In [19]:
%%time
model._init_all(interactions_train, user_features, item_features)

CPU times: user 604 ms, sys: 31.4 ms, total: 635 ms
Wall time: 676 ms


/Users/ericlundquist/anaconda3/lib/python3.7/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value


In [20]:
print(len(model.user_id), len(model.user_idx))
print(len(model.item_id), len(model.item_idx))

941 941
1421 1421


In [21]:
print(model.interactions.shape)
print(model.x_uf.shape)
print(model.x_if.shape)

(40598, 2)
(941, 26)
(1421, 18)


In [22]:
print("item weights: {}".format(model.w_i.shape))
print("item feature weights: {}".format(model.w_if.shape))
print("user factors: {}".format(model.v_u.shape))
print("item factors: {}".format(model.v_i.shape))
print("user feature factors: {}".format(model.v_uf.shape))
print("item feature factors: {}".format(model.v_if.shape))

item weights: (1421,)
item feature weights: (18,)
user factors: (941, 10)
item factors: (1421, 10)
user feature factors: (26, 10)
item feature factors: (18, 10)


In [23]:
model.interactions.dtype

dtype('int32')

In [24]:
uf_checker = pd.DataFrame(model.x_uf, columns=user_features.columns[1:])
uf_checker.head()

,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,occupation__engineer,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
user_features[user_features.user_id.isin(pd.Series([0,1,2,3,4]).map(model.index_to_user))]

,user_id,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,6,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
if_checker = pd.DataFrame(model.x_if, columns=item_features.columns[1:])
if_checker.head()

,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [27]:
item_features[item_features.item_id.isin(pd.Series([0,1,2,3,4]).map(model.index_to_item))]

,item_id,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [61]:
model = RankFM(factors=10, learning_rate=0.05, regularization=0.01, sigma=0.1)
model

#### Fit the Model on the Training Data and Profile Computational Performance

In [62]:
%%time

# model.fit(interactions_train, item_features=item_features, epochs=50, verbose=True)
model.fit(interactions_train, epochs=50, verbose=True)


training epoch: 0
log likelihood: -20974.13

training epoch: 1
log likelihood: -16778.31

training epoch: 2
log likelihood: -15751.05

training epoch: 3
log likelihood: -15211.62

training epoch: 4
log likelihood: -14972.9

training epoch: 5
log likelihood: -14530.26

training epoch: 6
log likelihood: -14145.38

training epoch: 7
log likelihood: -13596.94

training epoch: 8
log likelihood: -13147.14

training epoch: 9
log likelihood: -12518.97

training epoch: 10
log likelihood: -12120.51

training epoch: 11
log likelihood: -11539.4

training epoch: 12
log likelihood: -11153.27

training epoch: 13
log likelihood: -10698.64

training epoch: 14
log likelihood: -10396.48

training epoch: 15
log likelihood: -10011.57

training epoch: 16
log likelihood: -9944.38

training epoch: 17
log likelihood: -9681.63

training epoch: 18
log likelihood: -9662.77

training epoch: 19
log likelihood: -9325.2

training epoch: 20
log likelihood: -9192.56

training epoch: 21
log likelihood: -9140.98

traini

In [63]:
model.fit_partial(interactions_train, epochs=5, verbose=True)


training epoch: 0
log likelihood: -7597.81

training epoch: 1
log likelihood: -7511.53

training epoch: 2
log likelihood: -7534.39

training epoch: 3
log likelihood: -7589.35

training epoch: 4
log likelihood: -7461.77


#### Generate Model Scores

In [64]:
interactions_total.shape

(54194, 2)

In [65]:
%%time
scores = model.predict(interactions_total, cold_start='nan') 

CPU times: user 66.9 ms, sys: 3.34 ms, total: 70.2 ms
Wall time: 70 ms


#### Generate TopN Recommendations

In [66]:
all_users = pd.Series(interactions_total.user_id.unique())
print(all_users.shape)
all_users.head(10)

(943,)


0    253
1    286
2    200
3    224
4    122
5    291
6    119
7    167
8    299
9    308
dtype: int64

In [67]:
%%time
top_n = model.recommend_for_users(all_users, n_items=10, filter_previous=False, cold_start='nan')

CPU times: user 522 ms, sys: 10.2 ms, total: 532 ms
Wall time: 141 ms


In [68]:
top_n.head(10)

,0,1,2,3,4,5,6,7,8,9
253,69.0,204.0,64.0,423.0,174.0,50.0,22.0,98.0,127.0,210.0
286,1.0,382.0,83.0,268.0,419.0,393.0,246.0,451.0,168.0,173.0
200,174.0,79.0,69.0,265.0,172.0,181.0,210.0,98.0,161.0,385.0
224,328.0,258.0,313.0,300.0,155.0,15.0,332.0,66.0,69.0,245.0
122,132.0,191.0,135.0,197.0,357.0,216.0,86.0,423.0,196.0,655.0
291,410.0,288.0,11.0,763.0,200.0,219.0,217.0,56.0,173.0,1.0
119,237.0,50.0,181.0,117.0,121.0,258.0,1.0,222.0,742.0,111.0
167,511.0,197.0,485.0,705.0,483.0,514.0,661.0,435.0,132.0,86.0
299,654.0,603.0,285.0,483.0,197.0,514.0,517.0,135.0,474.0,134.0
308,603.0,185.0,135.0,98.0,56.0,483.0,173.0,89.0,200.0,654.0


#### Evaluate Model Performance on the Validation Data

In [69]:
interactions_valid.head()

,user_id,item_id
40646,437,443
40647,712,1037
40648,903,1070
40649,151,629
40650,711,79


In [70]:
k = 10

In [71]:
%%time
model_pre = round(precision_at_k(model, interactions_valid, k=k), 3)
model_rec = round(recall_at_k(model, interactions_valid, k=k), 3)

CPU times: user 2.26 s, sys: 23.8 ms, total: 2.29 s
Wall time: 705 ms


In [73]:
print("model precision: {} model recall: {}".format(model_pre, model_rec))

model precision: 0.127 model recall: 0.142


#### Evaluate Pure Popularity Baseline

In [74]:
most_popular = interactions_train.groupby('item_id').size().sort_values(ascending=False)[:k]
most_popular

item_id
50     357
100    291
181    277
258    260
98     249
174    246
127    245
1      225
56     223
313    221
dtype: int64

In [75]:
test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_recommends = model.recommend_for_users(list(test_user_items.keys()), cold_start='drop')
test_user_items = {key: val for key, val in test_user_items.items() if key in test_recommends.index}

base_pre = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()]), 3)
base_rec = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()]), 3)

print("number of test users: {}".format(len(test_user_items)))
print("baseline precision: {} baseline recall: {}".format(base_pre, base_rec))

number of test users: 903
baseline precision: 0.098 baseline recall: 0.088


#### Generate Model Scores for Test User/Items

In [76]:
mask = (interactions_valid.user_id.isin(interactions_train.user_id)) & (interactions_valid.item_id.isin(interactions_train.item_id))
print(interactions_valid.shape)
print(interactions_valid[mask].shape)

(13548, 2)
(13469, 2)


In [77]:
scores_miss = model.predict(interactions_valid, cold_start='nan')
scores_drop = model.predict(interactions_valid, cold_start='drop')

print(scores_miss.shape)
print(scores_drop.shape)

(13548,)
(13469,)


In [78]:
print(scores_miss.mean(), scores_miss.std(), scores_miss.min(), scores_miss.max())
print(scores_drop.mean(), scores_drop.std(), scores_drop.min(), scores_drop.max())

nan nan nan nan
1.5236446 1.0924941 -3.1793535 4.794591


#### Spot-Check Some User Recommendations

In [79]:
len(train_users), len(valid_users), cold_start_users

(941, 905, {2, 4})

In [80]:
recommendations = model.recommend_for_users(valid_users, n_items=10, filter_previous=True, cold_start='nan')
print(recommendations.shape)
recommendations.head(10)

(905, 10)


,0,1,2,3,4,5,6,7,8,9
1,172.0,475.0,179.0,216.0,357.0,180.0,11.0,483.0,204.0,273.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,333.0,302.0,313.0,288.0,682.0,750.0,258.0,289.0,300.0,326.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,68.0,210.0,202.0,96.0,588.0,91.0,181.0,184.0,195.0,174.0
6,603.0,654.0,285.0,657.0,661.0,204.0,663.0,705.0,190.0,659.0
7,357.0,69.0,173.0,705.0,211.0,202.0,419.0,493.0,197.0,143.0
8,234.0,64.0,265.0,96.0,98.0,69.0,117.0,204.0,135.0,121.0
9,302.0,269.0,313.0,515.0,268.0,100.0,340.0,127.0,242.0,258.0
10,661.0,427.0,511.0,479.0,98.0,474.0,180.0,199.0,513.0,657.0


In [81]:
recommendations[recommendations.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
user_id = 6
user_recs = recommendations.loc[user_id]
user_item_names = item_names[item_names.item_id.isin(user_recs)].set_index('item_id').loc[user_recs]
user_item_names

,item_name
item_id,
603.0,Rear Window (1954)
654.0,Chinatown (1974)
285.0,Secrets & Lies (1996)
657.0,"Manchurian Candidate, The (1962)"
661.0,High Noon (1952)
204.0,Back to the Future (1985)
663.0,Being There (1979)
705.0,Singin' in the Rain (1952)
190.0,Henry V (1989)


### Look at the Movies with the Highest/Lowest Model Weights

In [83]:
item_weights = pd.DataFrame({'item_idx': np.arange(len(model.w_i)), 'w_item': model.w_i})
item_weights['item_id'] = item_weights['item_idx'].map(model.index_to_item)
item_weights = pd.merge(item_weights, item_names, on='item_id').sort_values('w_item', ascending=False)
item_weights = item_weights[['item_id', 'item_idx', 'item_name', 'w_item']]

#### Best Movies Ever?

In [53]:
item_weights.head(10)

,item_id,item_idx,item_name,w_item
49,50,49,Star Wars (1977),2.110198
255,258,255,Contact (1997),2.075464
0,1,0,Toy Story (1995),2.055343
124,127,124,"Godfather, The (1972)",1.948900
99,100,99,Fargo (1996),1.907261
285,288,285,Scream (1996),1.852330
55,56,55,Pulp Fiction (1994),1.821599
283,286,283,"English Patient, The (1996)",1.810759
310,313,310,Titanic (1997),1.802520
266,269,266,"Full Monty, The (1997)",1.793087


#### Worst Movies Ever?

In [54]:
item_weights.tail(10)

,item_id,item_idx,item_name,w_item
1036,1076,1036,"Pagemaster, The (1994)",-1.636197
1418,1651,1418,"Spanish Prisoner, The (1997)",-1.640397
1157,1212,1157,Flesh and Bone (1993),-1.642887
1221,1290,1221,Country Life (1994),-1.650752
73,74,73,Faster Pussycat! Kill! Kill! (1965),-1.656521
1409,1632,1409,Land and Freedom (Tierra y libertad) (1995),-1.657477
1360,1515,1360,Wings of Courage (1995),-1.673015
1279,1393,1279,Stag (1997),-1.689040
1352,1501,1352,Prisoner of the Mountains (Kavkazsky Plennik) ...,-1.692659
1297,1416,1297,No Escape (1994),-1.698157


# Start Sandbox Code

In [ ]:
np.array_equal(sorted(x_if.index.values), self.item_idx)